In [ ]:
import os, json
import requests
import time, datetime
import base64
import pandas as pd
from IPython.core import display as ICD
import pprint as pp
from dotenv import load_dotenv
load_dotenv()

In [ ]:
BASE_URL = os.getenv("MYSHYN_URL")

TENANT_ID = os.getenv("MYSHYN_TENANT_ID")

USERNAME = os.getenv("MYSHYN_USERNAME")
PASSWORD = os.getenv("MYSHYN_PASSWORD")

REGISTRY_API_KEY = os.getenv("MYSHYN_REGISTRY_API")

PROXIES = {
    "https" : os.getenv("PROXIES_HTTPS")
}

In [ ]:
HEADER = {
    'Content-Type': 'application/json'
}

In [ ]:
def generateDateAndLast7day(date: None):
    # Get date
    selectedTime = datetime.datetime.strptime(date, '%d/%m/%y')
    last7dayTime = selectedTime - datetime.timedelta(days=6)
    
    # Convert date to timestamp
    selectedTime = selectedTime.strftime('%Y-%m-%dT23:59:00')
    last7dayTime = last7dayTime.strftime('%Y-%m-%dT00:00:00')

    # Convert timestamp to date
    # nowTime = datetime.datetime.fromtimestamp(nowTime)
    # last7dayTime = datetime.datetime.fromtimestamp(last7dayTime)
    
    return last7dayTime, selectedTime

In [ ]:
last7dayTime, selectedTime = generateDateAndLast7day(date='07/02/23')
last7dayTime, selectedTime

# Part 1 Executive Summary – Cloud Service Usage

In [ ]:
def getServiceCountStat(tenantId: None, selectedColumn: {}, tag: [], deviceHost: None, startDate: None, endDate: None):
    getServiceCountStatUrl = BASE_URL + '/shnapi/rest/reporting/csv/queryPerServiceCounts'
    print("URL  : " + getServiceCountStatUrl)
        
    BODY = {
        "tableVisibilityMap": selectedColumn,
        "eventFilter": {
            "tagIdentifiers": tag,
            "from":startDate,
            "to":endDate
        }
    }
    
    if deviceHost and deviceHost != []:
        BODY["eventFilter"]["customAttrDtos"] = [
            { "attributeColName":"custom1", "values": deviceHost }
        ] 
    
#     pp.pprint(BODY)
    
    r = requests.post(getServiceCountStatUrl, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), proxies=PROXIES, verify=False)
    
    if r.status_code == 200:
        return r.text
    else:
        print("[" + str(r.status_code) + "] Error : " + str(r.text))
        return None

In [ ]:
def generateSeletedColumns():
    return {
#         "Service Name" : True,
#         "Category" : True,
#         "Risk"  :True,
#         "Access Count" : True,
#         "Outbound" : True,
#         "Inbound" : True,
#         "Upload Count" : True,
#         "Upload Data" : True,
#         "Users" : True,
#         "Allowed" : True,
#         "Denied" : True
    }

In [ ]:
def extractDataToDataframe(data: None):
    res = [res.split(",") for res in data.split('\n')]
    res_df = pd.DataFrame(data=res[1:len(res)-1], columns=res[0]) #.dropna()
    
    del res
    
    columnToInt = ['Risk', 'Access Count', 'Outbound', 'Inbound', 'Upload Count', 'Upload Data', 'Users', 'Allowed', 'Denied', 'Allowed Data', 'Denied Data', 'Total Data']
    res_df[columnToInt] = res_df[columnToInt].apply(pd.to_numeric, errors='coerce')

    columnToDatetime = ['ServiceFirstUsed', 'ServiceLastUsed']
    res_df[columnToDatetime] = res_df[columnToDatetime].apply(pd.to_datetime, errors='coerce')
    
    return res_df

In [ ]:
def extractServiceByRisk(data: None):
    dataHigh_df = data[data["Risk"]>=7]
    dataMedium_df = data[(data["Risk"]>=4) & (data["Risk"]<7)]
    dataLow_df = data[(data["Risk"]>=0) & (data["Risk"]<4)]
    return dataHigh_df, dataMedium_df, dataLow_df

In [ ]:
def generateServiceStat(data: None, excludeService: None):
    CloudServices_df = extractDataToDataframe(data=data)
    
    # Exclude Allow Service
    CloudServices_df = CloudServices_df[~(CloudServices_df["Service Id"].isin(excludeService))]
    
    # Filter Allowed
    allowedServices_df = CloudServices_df[CloudServices_df["Allowed"]>0]
    
    # Filter Allowed with Risk
    allowwithHighRiskServices_df, allowwithMediumRiskServices_df, allowwithLowRiskServices_df = extractServiceByRisk(data=allowedServices_df)

    # Filter Allowed with Risk - CLoud Storage
    allowedCloudStorage_df = allowedServices_df[allowedServices_df["Category"]=="Cloud Storage"]
    allowwithHighRiskCloudStorage_df, allowwithMediumRiskCloudStorage_df, allowwithLowRiskCloudStorage_df = extractServiceByRisk(data=allowedCloudStorage_df)
    
    # Filter Denied
    deniedServices_df = CloudServices_df[CloudServices_df["Denied"]>0]

    return {
        "all" : CloudServices_df,
        "allowedServices" : {
            "all" : allowedServices_df,
            "highRisk" : allowwithHighRiskServices_df,
            "mediumRisk" : allowwithMediumRiskServices_df,
            "lowRisk" : allowwithLowRiskServices_df,
            "cloudStorage" : {
                "all" : allowedCloudStorage_df,
                "highRisk" : allowwithHighRiskCloudStorage_df,
                "mediumRisk" : allowwithMediumRiskCloudStorage_df,
                "lowRisk" : allowwithLowRiskCloudStorage_df,
            }
        },
        "deniedServices" : deniedServices_df
    }

In [ ]:
def printServiceStat(allData: None, allowedData: None, deniedData: None):
    print("\nTotal Service Statistic")                                                          
    print("All Cloud Service Discovered : %d services" % (len(allData)))
    print("All Cloud Service Allowed Traffic : %d services" % (len(allowedData["all"])))
    print("All Cloud Service Allowed Traffic by Risk : %d(%d,%d,%d) services" % (len(allowedData["all"]),
                                                                                 len(allowedData["highRisk"]),
                                                                                 len(allowedData["mediumRisk"]),
                                                                                 len(allowedData["lowRisk"])))
    print("Cloud Storage Services Allowed Traffic : %d(%d,%d,%d) services" % (len(allowedData["cloudStorage"]["all"]),
                                                                              len(allowedData["cloudStorage"]["highRisk"]),
                                                                              len(allowedData["cloudStorage"]["mediumRisk"]),
                                                                              len(allowedData["cloudStorage"]["lowRisk"])))
    print("All Cloud Service Denied Traffic : %d services" % (len(deniedData)))
    

In [ ]:
fp_tag = 0
pa_tag = 0
wah_tag = 0

AllowServiceCSPIDList = ["0"]

# Device Host // Custom 1 Tag
remote_type = ["A"]
oversea_type = ["A"]

selectedColumn = generateSeletedColumns()

In [ ]:
# Forcepoint Proxy
fpServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[fp_tag],
                                    deviceHost=None,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
fpServiceCounts = generateServiceStat(data=fpServiceCounts, excludeService=AllowServiceCSPIDList)

# Remote Access
rmServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[pa_tag],
                                    deviceHost=remote_type,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
rmServiceCounts = generateServiceStat(data=rmServiceCounts, excludeService=AllowServiceCSPIDList)

# Oversea Branch
ozServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[pa_tag],
                                    deviceHost=oversea_type,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
ozServiceCounts = generateServiceStat(data=ozServiceCounts, excludeService=AllowServiceCSPIDList)

# Work Anywhere, Skyhigh
wawServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[wah_tag],
                                    deviceHost=None,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
wawServiceCounts = generateServiceStat(data=wawServiceCounts, excludeService=AllowServiceCSPIDList)


In [ ]:
print("\nPart 1 Executive Summary – Cloud Service Usage : ")

print("\nHQ : ")
printServiceStat(allData=fpServiceCounts["all"],
                 allowedData=fpServiceCounts["allowedServices"],
                 deniedData=fpServiceCounts["deniedServices"])
ICD.display(fpServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nRemote Access : ")
printServiceStat(allData=rmServiceCounts["all"],
                 allowedData=rmServiceCounts["allowedServices"],
                 deniedData=rmServiceCounts["deniedServices"])
ICD.display(rmServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nOversea Branch: ")
printServiceStat(allData=ozServiceCounts["all"],
                 allowedData=ozServiceCounts["allowedServices"],
                 deniedData=ozServiceCounts["deniedServices"])
ICD.display(ozServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nWork Anywhere Gateway : ")
printServiceStat(allData=wawServiceCounts["all"],
                 allowedData=wawServiceCounts["allowedServices"],
                 deniedData=wawServiceCounts["deniedServices"])
ICD.display(wawServiceCounts["allowedServices"]["highRisk"].head(5))

# Part 2 Anomalies Activities UEBA

Go to portal and Capture Image

In [ ]:
last7dayTime, selectedTime

In [ ]:
# Anomalies
# Status = new, Service Name = Exchange, Onedrive, Sharepoint
# Capture Image

{
    "type":"in_search_query",
    "field":"incident_detail.services.name",
    "values":["sharepoint","onedrive","exchange online"]
},
{
  "type": "equal_search_query",
  "value": "new",
  "field": "workflow.status"
}
{
    "type":"equal_search_query",
    "value":"2",
    "field":"severity"
}

In [ ]:
# hqAllCloudServices_df[["Service Id", "Service Name"]][hqAllCloudServices_df["Service Name"].str.contains("microsoft", case=False)]

In [ ]:
# eventFilter = {
# #     "serviceidentifiers": ["2799", "3210"],
#     "from":last7dayTime,
#     "to":nowTime,
# }

In [ ]:
# BODY = {
#     "templateId" : 10,
#     "eventFilter" : eventFilter
# }

# BODY

In [ ]:
# url = BASE_URL + '/shnapi/rest/reporting/csv/queryAnomaliesForUser'
# print("URL  : " + url)
# r = requests.post(url, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), verify=False)
# r.status_code

In [ ]:
# res = [res.split(",") for res in r.text.split('\n')]
# res_df = pd.DataFrame(data=res[1:], columns=res[0])#.dropna()
# res_df

# Part 3 - Activities

In [ ]:
# Incidents > User Activity > Activities
# Activity Type

# "sharepoint","onedrive","exchange online"

In [ ]:
url = BASE_URL + '/shnapi/rest/external/api/v1/queryIncidents?limit=500'
print("URL  : " + url)

BODY =  { 
    "startTime": datetime.datetime.now().date().strftime('%Y-%m-%dT00:00:00'), #last7dayTime,
    "serviceNames":["sharepoint"] # ,"onedrive","exchange"
#     "incidentCriteria": { 
#         "categories": [ 
#             {
#                 "incidentType":"Threat"
#             }
#         ] 
#     } 
} 

BODY

In [ ]:
# r = requests.post(url, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), verify=False)
# r.status_code, r.json()

In [ ]:
allresult = []

In [ ]:
while True:
    print(BODY)
    r = requests.post(url, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), verify=False)
    print(r.status_code)
    
    res = r.json()["body"]
    
    allresult += res["incidents"]
    
    print(res["responseInfo"]["nextStartTime"])
    BODY["startTime"] = res["responseInfo"]["nextStartTime"]
    if res["responseInfo"]["nextStartTime"] != None and res["incidents"] != []:
        BODY["startTime"] = res["responseInfo"]["nextStartTime"]
#         print(BODY["startTime"])
#         break
    else:
        print(res["responseInfo"]["nextStartTime"])
        break
    

In [ ]:
allresult

In [ ]:
resincidents_df = pd.DataFrame(allresult)#.dropna()
resincidents_df

In [ ]:
list(resincidents_df["activityNames"].unique())